In [1]:
import os
import sys
from pathlib import Path
import logging
import numpy as np
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

In [2]:
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  # If notebooks are in langechain/notebooks

# Add project root to Python path for imports
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project root: {project_root}")

Project root: /app


In [3]:
from src.data_pipline.json_loader.json_loader_epo import EPOPatentLoader
from config.json_loader_config import PatentLoaderConfig
from src.embeddings.text_Chunker import TextChunker, ChunkerConfig


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("patent_pipeline")

In [6]:
data_dir = Path(project_root) / "data" / "parsed" / "EPO"
model_name = "sentence-transformers/all-mpnet-base-v2"
vector_db_dir = Path(project_root) / "data" / "vector_db"
vector_db_dir.mkdir(parents=True, exist_ok=True)

In [7]:
loader_config = PatentLoaderConfig(
    max_files=200,
    
    verbose=False,
    epo_dir=data_dir,
    include_abstract=True,
    include_claims=True,
    include_description=True
)
patent_loader = EPOPatentLoader(config=loader_config)
print("✅ Patent loader initialized")

✅ Patent loader initialized


In [17]:
json_files = patent_loader.load_raw_json_files()

2025-06-19 22:34:34,057 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 200 files out of 200 total files
2025-06-19 22:34:34,057 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 200 files out of 200 total files
2025-06-19 22:34:34,058 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 200 JSON files in /app/data/parsed/EPO
2025-06-19 22:34:34,058 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 200 JSON files in /app/data/parsed/EPO
2025-06-19 22:34:34,098 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 200 raw JSON files
2025-06-19 22:34:34,098 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 200 raw JSON files


In [18]:
documents  = patent_loader.extract_document_components()

2025-06-19 22:34:35,034 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 200 files out of 200 total files
2025-06-19 22:34:35,034 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 200 files out of 200 total files
2025-06-19 22:34:35,035 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 200 JSON files in /app/data/parsed/EPO
2025-06-19 22:34:35,035 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 200 JSON files in /app/data/parsed/EPO
2025-06-19 22:34:35,169 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 199 documents from 200 files
2025-06-19 22:34:35,169 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 199 documents from 200 files


In [19]:
json_files = []

In [20]:
print(f"✅ Loaded {len(documents)} document segments from {len(json_files)} patent files")


✅ Loaded 199 document segments from 0 patent files


In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
chunker_config = ChunkerConfig(
    base_chunk_size=300,  # Adjust based on your needs
    chunk_overlap=30,
    min_chunk_size=100
)
chunker = TextChunker(tokenizer, config=chunker_config)
print("✅ TextChunker initialized")

✅ TextChunker initialized


In [22]:
print("✂️ Chunking documents...")
chunked_docs = []

for docs in tqdm(documents, desc="Chunking"):
    for doc in tqdm(docs, desc="Chunking"):
        try:
            text = doc.page_content
            metadata = doc.metadata
            
            # Use your TextChunker to split the document
            chunks = chunker.chunk_text(text, use_semantic=True)
            
            # Create LangChain Document objects from chunks
            for i, chunk_text in enumerate(chunks):
                # Copy metadata and add chunk information
                chunk_metadata = metadata.copy()
                chunk_metadata.update({
                    "chunk_index": i,
                    "total_chunks": len(chunks)
                })
                
                chunked_docs.append(Document(
                    page_content=chunk_text,
                    metadata=chunk_metadata
                ))
        except Exception as e:
            logger.error(f"Error chunking document: {str(e)}")

✂️ Chunking documents...


Chunking:   0%|          | 0/199 [00:00<?, ?it/s]

Chunking:   0%|          | 0/180 [00:00<?, ?it/s]

2025-06-19 22:34:42,333 - src.embeddings.text_Chunker - INFO - Input text has 5 tokens
2025-06-19 22:34:42,334 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:42,335 - src.embeddings.text_Chunker - INFO - Input text has 33 tokens
2025-06-19 22:34:42,336 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:42,336 - src.embeddings.text_Chunker - INFO - Input text has 33 tokens
2025-06-19 22:34:42,339 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:42,340 - src.embeddings.text_Chunker - INFO - Input text has 116 tokens
2025-06-19 22:34:42,341 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:42,342 - src.embeddings.text_Chunker - INFO - Input text has 37 tokens
2025-06-19 22:34:42,342 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:42,343 - src.embeddings.text_Chunker - INFO - Input text has 88 tokens
2025-06-19 22:34:42,344 - s

Chunking:   0%|          | 0/16 [00:00<?, ?it/s]

...

2025-06-19 22:34:53,160 - src.embeddings.text_Chunker - INFO - Input text has 15 tokens
2025-06-19 22:34:53,160 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,161 - src.embeddings.text_Chunker - INFO - Input text has 273 tokens
2025-06-19 22:34:53,161 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,162 - src.embeddings.text_Chunker - INFO - Input text has 31 tokens
2025-06-19 22:34:53,163 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,163 - src.embeddings.text_Chunker - INFO - Input text has 16 tokens
2025-06-19 22:34:53,163 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,164 - src.embeddings.text_Chunker - INFO - Input text has 37 tokens
2025-06-19 22:34:53,164 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,164 - src.embeddings.text_Chunker - INFO - Input text has 81 tokens
2025-06-19 22:34:53,165 - 

Chunking:   0%|          | 0/44 [00:00<?, ?it/s]

2025-06-19 22:34:53,204 - src.embeddings.text_Chunker - INFO - Input text has 12 tokens
2025-06-19 22:34:53,204 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,204 - src.embeddings.text_Chunker - INFO - Input text has 33 tokens
2025-06-19 22:34:53,205 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,205 - src.embeddings.text_Chunker - INFO - Input text has 122 tokens
2025-06-19 22:34:53,206 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,206 - src.embeddings.text_Chunker - INFO - Input text has 113 tokens
2025-06-19 22:34:53,207 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,207 - src.embeddings.text_Chunker - INFO - Input text has 131 tokens
2025-06-19 22:34:53,207 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,208 - src.embeddings.text_Chunker - INFO - Input text has 116 tokens
2025-06-19 22:34:53,208

Chunking:   0%|          | 0/350 [00:00<?, ?it/s]

2025-06-19 22:34:53,248 - src.embeddings.text_Chunker - INFO - Input text has 18 tokens
2025-06-19 22:34:53,248 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,248 - src.embeddings.text_Chunker - INFO - Input text has 34 tokens
2025-06-19 22:34:53,249 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,250 - src.embeddings.text_Chunker - INFO - Input text has 73 tokens
2025-06-19 22:34:53,250 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,250 - src.embeddings.text_Chunker - INFO - Input text has 105 tokens
2025-06-19 22:34:53,252 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,252 - src.embeddings.text_Chunker - INFO - Input text has 35 tokens
2025-06-19 22:34:53,252 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-19 22:34:53,253 - src.embeddings.text_Chunker - INFO - Input text has 56 tokens
2025-06-19 22:34:53,253 - 

In [23]:
documents = []
chunked_docs[0]

Document(metadata={'doc_id': 'EP13899497B9W1', 'language': 'en', 'country': 'EP', 'doc_number': '3084761', 'application_number': '13899497.5', 'publication_date': '20250611', 'ipc_classes': 'G10L  19/038       20130101AFI20170426BHEP, G10L  19/07        20130101ALI20170426BHEP', 'file': 'EP13899497W1B9.xml', 'filePath': '/app/data/parsed/EPO/EPRTBJV2025000024001001/EPW1B9/EP13899497W1B9/EP13899497W1B9.json', 'title': 'AUDIO SIGNAL ENCODER', 'section': 'title', 'chunk_index': 0, 'total_chunks': 1}, page_content='AUDIO SIGNAL ENCODER')

In [24]:
print(f"✅ Created {len(chunked_docs)} chunks from {len(documents)} document segments")


✅ Created 17428 chunks from 0 document segments


In [25]:

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"batch_size": 32, "normalize_embeddings": True}
)
print("✅ LangChain embeddings initialized")


/tmp/ipykernel_3966/3069164953.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2025-06-19 22:34:55,595 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


✅ LangChain embeddings initialized


In [26]:
# 7. Create vector store with chunked documents - connect to Docker ChromaDB
print("💾 Creating vector store using dockerized ChromaDB...")

# Import ChromaDB client directly (needed for Docker connection)
import chromadb
from chromadb.config import Settings

# When running inside Docker
chroma_client = chromadb.HttpClient(
    host="vector_db",  # Docker service name
    port=8000,         # Internal container port
    settings=Settings(anonymized_telemetry=False)
)

💾 Creating vector store using dockerized ChromaDB...


2025-06-19 22:34:57,467 - httpx - INFO - HTTP Request: GET http://vector_db:8000/api/v2/auth/identity "HTTP/1.1 200 OK"
2025-06-19 22:34:57,495 - httpx - INFO - HTTP Request: GET http://vector_db:8000/api/v2/tenants/default_tenant "HTTP/1.1 200 OK"
2025-06-19 22:34:57,498 - httpx - INFO - HTTP Request: GET http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database "HTTP/1.1 200 OK"


In [27]:
collection_name = "patents"

vectorstore = Chroma(
    client=chroma_client,
    collection_name=collection_name,
    embedding_function=embeddings
)

/tmp/ipykernel_3966/978458448.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
2025-06-19 22:35:01,431 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections "HTTP/1.1 200 OK"


In [29]:
# Add documents to the vectorstore (if we have any)
if chunked_docs:
    # Add in smaller batches to avoid memory issues
    batch_size = 100
    for i in tqdm(range(0, len(chunked_docs), batch_size), desc="Adding to vector store"):
        batch = chunked_docs[i:i+batch_size]
        vectorstore.add_documents(documents=batch)
    
    print(f"✅ Added {len(chunked_docs)} document chunks to ChromaDB collection '{collection_name}'")
else:
    print("⚠️ No document chunks to add to the vector store!")

Adding to vector store:   0%|          | 0/175 [00:00<?, ?it/s]

2025-06-19 22:35:50,142 - httpx - INFO - HTTP Request: GET http://vector_db:8000/api/v2/pre-flight-checks "HTTP/1.1 200 OK"
2025-06-19 22:35:50,250 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/ed371e4e-de10-48d9-84f5-ceb6d5b769cd/upsert "HTTP/1.1 200 OK"
2025-06-19 22:35:56,690 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/ed371e4e-de10-48d9-84f5-ceb6d5b769cd/upsert "HTTP/1.1 200 OK"
2025-06-19 22:36:05,187 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/ed371e4e-de10-48d9-84f5-ceb6d5b769cd/upsert "HTTP/1.1 200 OK"
2025-06-19 22:36:13,551 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/ed371e4e-de10-48d9-84f5-ceb6d5b769cd/upsert "HTTP/1.1 200 OK"
2025-06-19 22:36:22,098 

✅ Added 17428 document chunks to ChromaDB collection 'patents'


In [ ]:
# 8. Test retrieval (only if we have documents)
if chunked_docs:
    print("\n🔍 Testing vector store with sample queries...")

    test_queries = [
        "Audio signal encoding methods",
        "Image compression techniques",
        "Wireless communication protocols"
    ]

    for query in test_queries:
        print(f"\nQuery: '{query}'")
        # Get top 3 results
        results = vectorstore.similarity_search_with_score(query, k=3)
        
        print(f"Top {len(results)} results:")
        for i, (doc, score) in enumerate(results):
            print(f"\nResult {i+1} (similarity: {score:.4f})")
            print(f"Title: {doc.metadata.get('title', 'N/A')}")
            print(f"Section: {doc.metadata.get('section', 'N/A')}")
            print(f"Patent ID: {doc.metadata.get('doc_id', 'N/A')}")
            print(f"Content preview: {doc.page_content[:150]}...")
else:
    print("\n⚠️ No documents in vector store to search!")

In [30]:
vectorstore.delete_collection()

2025-06-19 22:04:58,996 - httpx - INFO - HTTP Request: DELETE http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/patents "HTTP/1.1 200 OK"
